# Aufgabe 1 - Prozessorientierte Layoutgestaltung

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	\min\ \ Z = \sum_{h=1}^{p}\sum_{\stackrel{i=1}{ i\ne h}}^{p}\sum_{j=1}^{p}\sum_{\stackrel{k=1}{k\ne j} }^{p} t_{hi} \cdot d_{jk} \cdot x_{hj} \cdot x_{ik}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&&  \sum_{j=1}^{p}x_{hj} &= 1 && \forall h \\[5pt]
&&  \sum_{h=1}^{p}x_{hj} &= 1 && \forall j \\[10pt]
&& x_{hj} &\in \{0,1\} &&
\end{align}


## Aufgabe 1b)
 Lesen Sie die beiden Dateien `Distanz.csv` und `Transport.csv` ein.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import csv

### Distanz.csv

|  |  |  |  |  |  |
|---|---|---|---|---|---|
| 0 | 4 | 7 | 3 | 4 | 8 |
| 4 | 0 | 3 | 5 | 8 | 4 |
| 7 | 3 | 0 | 4 | 7 | 3 |
| 3 | 5 | 4 | 0 | 3 | 5 |
| 4 | 8 | 7 | 3 | 0 | 4 |
| 8 | 4 | 3 | 5 | 4 | 0 |

In [ ]:
distance=[]
with open("Distanz.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row]
          distance.append(rowAsInt)

### Transport.csv

| Ort        | Kapazität | Fixkosten | 
|:-----------|-----------|-----------| 
| Dresden    | 400       | 60000     | 
| Bremen     | 350       | 60000     | 
| Düsseldorf | 600       | 80000     | 

In [ ]:
transportAmount=[]
with open("Transport.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row]
          transportAmount.append(rowAsInt)

### Definition der Mengen

In [ ]:
numberOfMaschines = len(transportAmount) # oder händisch: 6
maschines = range(numberOfMaschines)
numberOfPlaces = len(distance) # oder händisch: 6
places = range(numberOfPlaces)

## Aufgabe 4d)
Lassen Sie sich zur Kontrolle jede Zeile der berechneten Kostenmatrix in der Konsole ausgeben und lösen Sie das Problem.

In [ ]:
for row in c:
    print(row)

Initialisierung des Modells:

In [ ]:
m = gp.Model()

Initialisierung der Variablen:

In [ ]:
x = {}
for i in I:
     for j in J:
          x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i)+str(j))

In [ ]:
y = {}
for i in I:
     y[i] = m.addVar(vtype=GRB.BINARY, name="y_"+str(i))

Definition der Zielfunktion:

In [ ]:
m.setObjective(gp.quicksum(f[i] * y[i] for i in I) + gp.quicksum(c[i][j] * x[i,j] for j in J for i in I), GRB.MINIMIZE)

Hinzufügen der Nebenbedingungen:

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) == d[j], "meet_demand_" + str(j))

In [ ]:
for i in I:
     m.addConstr(gp.quicksum(x[i,j] for j in J) <= b[i] * y[i], "meet_prod_" + str(i))

Optimierung:

In [ ]:
m.optimize()

## Aufgabe 4e)
Schreiben Sie in die Konsole als Vorbereitung eines aussagekräftigen Ergebnisses den Satz: "Optimale Standortwahl bei mehreren Betriebsstätten".

In [ ]:
print("\nOptimale Standortwahl bei mehreren Betriebsstätten\n")

## Aufgabe 4f)
Geben Sie den Zielfunktionswert innerhalb eines Satzes an.

In [ ]:
print("Die gesamten Kosten des Transportes betragen", m.getAttr(GRB.Attr.ObjVal), "GE.")

## Aufgabe 4g)
Formulieren Sie anschließend die Standortwahl.

In [ ]:
for i in I:
     if y[i].X > 0:
          print("In", so[i], "wird ein Standort errichtet.")
          print("\tFixkosten:", f[i], "GE")
          print("\tVariable Kosten:", sum([c[i][j] * x[i,j].X for j in J]), "GE")
     else:
          print("In", so[i], "wird KEIN Standort errichtet.")

## Aufgabe 4h)
Des Weiteren sollen die Lieferungen zwischen den Standorten sowie die jeweils zugehörigen Transportkosten in der Konsole beschrieben werden.

In [ ]:
for i in I:
     for j in J:
          if x[i,j].X > 0:
               print("Von", so[i], "werden", x[i,j].X, "ME nach", ao[j], "geliefert, die Transportkosten betragen dabei", c[i][j] * x[i,j].X, "GE.")

## Aufgabe 4i)
Erzeugen Sie eine Textdatei "Ergebnisausgabe.txt" und schreiben Sie in diese Datei die Ausgabe von e) -- h).

In [ ]:
with open("Ergebnisausgabe.txt", "w", encoding="utf-8") as file:
     file.write("Optimale Standortwahl bei mehreren Betriebsstätten\n\n")
     file.write("Die gesamten Kosten des Transportes betragen " + str(m.getAttr(GRB.Attr.ObjVal)) + " GE.\n")
     for i in I:
          if y[i].X > 0:
               file.write("In " + so[i] + " wird ein Standort errichtet!\n")
               file.write("\tFixkosten: " + str(f[i]) + "GE \n")
               file.write("\tVariable Kosten: " + str(sum([c[i][j] * x[i,j].X for j in J])) + " GE\n")
          else:
               file.write("In " + so[i] + " wird KEIN Standort errichtet!\n")
     file.write("\n")
     for i in I:
          for j in J:
               if x[i,j].X > 0:
                    file.write("Von " + so[i] + " werden " + str(x[i,j].X) + " ME nach " + ao[j] +
                    " geliefert, die Transportkosten betragen dabei " + str(c[i][j] * x[i,j].X) + " GE.\n")